## 1. Import

In [2]:
import numpy as np 
import pandas as pd
import matplotlib as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## 2. Đọc dữ liệu đã qua tiền xử lý cơ bản

In [3]:
eda_gas_df = pd.read_csv("data/PROCESSED_heating-gas-consumption-and-cost.csv")
eda_gas_df

,Development Name,Borough,Location,Funding Source,Vendor Name,Revenue Month,Service Start Date,Service End Date,# days,Current Charges,Consumption (Therms)
0,ADAMS,BRONX,BLD 04,FEDERAL,CONSOLIDATED EDISON COMPANY OF NY,2010-01-01,2009-12-24,2010-01-26,33,78292.97,136632.00
1,ALBANY/ALBANY II,BROOKLYN,ALBANY BLD 04,FEDERAL,NATIONAL GRID NYC,2010-01-01,2009-12-29,2010-01-28,30,44335.21,153899.18
2,ALBANY/ALBANY II,BROOKLYN,ALBANY BLD 04,FEDERAL,NATIONAL GRID NYC,2010-01-01,2009-12-29,2010-01-28,30,22034.86,76488.82
3,AMSTERDAM,MANHATTAN,BLD 02,FEDERAL,CONSOLIDATED EDISON COMPANY OF NY,2010-01-01,2009-12-24,2010-01-26,33,95456.57,169137.00
4,AMSTERDAM AVENUE,MANHATTAN,BLD 01,FEDERAL,CONSOLIDATED EDISON COMPANY OF NY,2010-01-01,2009-12-24,2010-01-26,33,14607.73,26301.00
...,...,...,...,...,...,...,...,...,...,...,...
92194,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,NATIONAL GRID NYC,2021-06-01,2021-05-25,2021-06-25,31,1419.26,3615.00
92195,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,NATIONAL GRID NYC,2021-10-01,2021-09-24,2021-10-29,35,1036.26,1619.00
92196,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,NATIONAL GRID NYC,2021-11-01,2021-10-29,2021-12-01,33,2505.80,6768.00
92197,WYCKOFF GARDENS,BROOKLYN,BLD 03,FEDERAL,NATIONAL GRID NYC,2021-12-01,2021-12-21,2022-01-06,16,20992.58,50105.00


## 3. Khám phá và các kỹ thuật chuẩn hóa dữ liệu

### Trước hết, xem lại kiểu dữ liệu sau khi đọc

In [4]:
eda_gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92199 entries, 0 to 92198
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Development Name      92199 non-null  object 
 1   Borough               92199 non-null  object 
 2   Location              92199 non-null  object 
 3   Funding Source        92199 non-null  object 
 4   Vendor Name           92199 non-null  object 
 5   Revenue Month         92199 non-null  object 
 6   Service Start Date    92199 non-null  object 
 7   Service End Date      92199 non-null  object 
 8   # days                92199 non-null  int64  
 9   Current Charges       92199 non-null  float64
 10  Consumption (Therms)  92199 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 7.7+ MB


- Các cột dữ liệu về `Revenue Month`, `Service Start Date`, `Service End Date` phải được chuyển về dạng datetime

In [5]:
eda_gas_df['Revenue Month'] = pd.to_datetime(eda_gas_df['Revenue Month'], format="%Y-%m")
eda_gas_df['Service Start Date'] = eda_gas_df['Service Start Date'].astype("datetime64")
eda_gas_df['Service End Date'] = eda_gas_df['Service End Date'].astype("datetime64")

- Xem lại hàm info(), các kiểu dữ liệu của các cột nhìn chung đã hợp lý

In [6]:
eda_gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92199 entries, 0 to 92198
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Development Name      92199 non-null  object        
 1   Borough               92199 non-null  object        
 2   Location              92199 non-null  object        
 3   Funding Source        92199 non-null  object        
 4   Vendor Name           92199 non-null  object        
 5   Revenue Month         92199 non-null  datetime64[ns]
 6   Service Start Date    92199 non-null  datetime64[ns]
 7   Service End Date      92199 non-null  datetime64[ns]
 8   # days                92199 non-null  int64         
 9   Current Charges       92199 non-null  float64       
 10  Consumption (Therms)  92199 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int64(1), object(5)
memory usage: 7.7+ MB


### a. Các cột categotical

In [7]:
category_df = eda_gas_df.loc[:, ['Development Name', 'Borough', 'Location', 'Funding Source',  'Vendor Name']]

#### Có chứa giá trị abnormal bất thường không?
- Trong categorical column, abnormal là một giá trị không phù hợp với phần còn lại của dữ liệu trong cột đó.
- Một số ví dụ cụ thể về abnormal trong categorical column:

  + Một giá trị xuất hiện rất ít trong dữ liệu. Ví dụ, trong cột "Trạng thái", nếu giá trị "Đang chờ" xuất hiện chỉ 1 lần trong số 1000 giá trị, thì giá trị này có thể được coi là abnormal.
  + Một giá trị không có ý nghĩa trong ngữ cảnh của dữ liệu. Ví dụ, trong cột "Ngôn ngữ", nếu giá trị "Không có" xuất hiện trong một tập dữ liệu về các ngôn ngữ được sử dụng, thì giá trị này có thể được coi là abnormal.
  + Một giá trị bị lỗi hoặc nhập sai. Ví dụ, trong cột "Mã sản phẩm", nếu giá trị "ABC123456789" xuất hiện, thì giá trị này có thể được coi là abnormal vì nó vượt quá số lượng ký tự tối đa cho phép.

- Trong bài, xác định abnormal bằng phương pháp tần suất: tìm các giá trị xuất hiện ít hơn một ngưỡng nhất định, ví dụ như dưới ngưỡng 5.

In [8]:
# Tìm các giá trị xuất hiện ít hơn 2 lần
for col in category_df.columns:
    check = True
    print("Giá trị bất thường của cột", col, ": ", end = "")
    for value in category_df[col].value_counts().index:
        if category_df[col].value_counts()[value] < 5:
            check = False
            print(value, end = ", ")
    if check == True:
        print("Không có", end="")
    print()


Giá trị bất thường của cột Development Name : WSUR (SITE B) 74 WEST 92ND STREET, 1471 WATSON AVENUE, 
Giá trị bất thường của cột Borough : Không có
Giá trị bất thường của cột Location : Không có
Giá trị bất thường của cột Funding Source : Không có
Giá trị bất thường của cột Vendor Name : DIRECT ENERGY BUSINESS (32179), 


- Như vậy, các cột `Development Name`, `Vendor Name` có chứa giá trị bất thường (abnormal). 

#### Encoding the categorical columns

- Các cột categorical cần được encoded (mã hóa) để có thể được sử dụng bởi các mô hình học máy. Các mô hình học máy, chẳng hạn như các mô hình hồi quy, phân loại, và clustering, được thiết kế để làm việc với các dữ liệu số. Các cột categorical, mặt khác, chứa các giá trị văn bản hoặc ký tự. Để các mô hình học máy có thể hiểu được các cột categorical, chúng cần được chuyển đổi thành các giá trị số.
- Một số cách phổ biến khác nhau để encode các cột categorical như: one-hot encoding, dummy encoding, label encoding, count encoding.

**One-hot encoding**

- One-hot encoding là một kỹ thuật biến đổi dữ liệu được sử dụng để chuyển đổi các giá trị phân loại thành các giá trị nhị phân. Mỗi giá trị phân loại được đại diện bởi một vector có số chiều bằng với số lượng giá trị phân loại khả thi. Mỗi chiều của vector này sẽ có giá trị 1 nếu giá trị phân loại tương ứng với giá trị phân loại hiện tại và ngược lại là giá trị 0.
- One-hot encoding là một kỹ thuật cơ bản nhưng quan trọng trong học máy. Nó được sử dụng rộng rãi trong nhiều dạng khác nhau, chẳng hạn như phân loại, hồi quy, và clustering.

In [9]:
one_hot_df = pd.get_dummies(category_df)
one_hot_df

,Development Name_1010 EAST 178TH STREET,Development Name_104-14 TAPSCOTT STREET,Development Name_1162-1176 WASHINGTON AVENUE,Development Name_131 SAINT NICHOLAS AVENUE,Development Name_1471 WATSON AVENUE,Development Name_154 WEST 84TH STREET,Development Name_303 VERNON AVENUE,Development Name_335 EAST 111TH STREET,Development Name_45 ALLEN STREET,Development Name_572 WARREN STREET,...,Location_WEST BRIGHTON I BLD 06,Funding Source_FEDERAL,Funding Source_FEDERAL-COOP,Funding Source_MIXED FINANCE/LLC1,Funding Source_MIXED FINANCE/LLC2,Funding Source_NON-DEVELOPMENT,Vendor Name_CONSOLIDATED EDISON COMPANY OF NY,Vendor Name_DIRECT ENERGY BUSINESS (32179),Vendor Name_NATIONAL GRID LI,Vendor Name_NATIONAL GRID NYC
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92194,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
92195,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
92196,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
92197,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


- Ưu điểm của One-hot encoding:
  + Có khả năng tương thích với các mô hình học máy, giúp chuyển đổi các giá trị phân loại thành các dữ liệu số, từ đó các mô hình học máy có thể hiểu và xử lý các giá trị này.
  + Giảm thiểu overfitting, vì nó giúp các mô hình học máy tập trung vào các giá trị phân loại quan trọng.
- Nhược điểm:
  + Tăng kích thước dữ liệu vì mỗi giá trị phân loại sẽ được đại diện bởi một vector nhị phân, dẫn đến việc tăng chi phí lưu trữ và tính toán.
  + Có thể làm mất thông tin về thứ tự của các giá trị phân loại. Ví dụ, vector nhị phân cho giá trị "X" giống hệt với vector nhị phân cho giá trị "Y".

**Dummy encoding**

- Dummy encoding là một kỹ thuật chuyển đổi các giá trị phân loại thành các biến giả. Mỗi giá trị phân loại sẽ được đại diện bởi một biến giả, trong đó biến giả có giá trị 1 nếu giá trị phân loại là giá trị đó và giá trị 0 nếu ngược lại, sử dụng N - 1 tính năng để thể hiện N (toàn bộ) nhãn/danh mục/cột dữ liệu.
- Giống như các encoding, nó có ý nghĩa quan trọng trong học máy, vì nó giúp các mô hình học máy có thể hiểu và xử lý các giá trị phân loại.

In [10]:
dummy_df = pd.get_dummies(category_df, drop_first=True)
dummy_df

,Development Name_104-14 TAPSCOTT STREET,Development Name_1162-1176 WASHINGTON AVENUE,Development Name_131 SAINT NICHOLAS AVENUE,Development Name_1471 WATSON AVENUE,Development Name_154 WEST 84TH STREET,Development Name_303 VERNON AVENUE,Development Name_335 EAST 111TH STREET,Development Name_45 ALLEN STREET,Development Name_572 WARREN STREET,Development Name_830 AMSTERDAM AVENUE,...,Location_VAN DYKE I BLD 12,Location_VLADECK BLD 02,Location_WEST BRIGHTON I BLD 06,Funding Source_FEDERAL-COOP,Funding Source_MIXED FINANCE/LLC1,Funding Source_MIXED FINANCE/LLC2,Funding Source_NON-DEVELOPMENT,Vendor Name_DIRECT ENERGY BUSINESS (32179),Vendor Name_NATIONAL GRID LI,Vendor Name_NATIONAL GRID NYC
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
92195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
92196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
92197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


- Ưu điểm của dummy encoding:
  + Tương thích với các mô hình học máy bằng cách chuyển đổi các giá trị phân loại thành các dữ liệu số, giúp các mô hình học máy có thể hiểu và xử lý các giá trị này.
  + So với one-hot, dummy encoding không làm tăng kích thước dữ liệu, vì mỗi giá trị phân loại sẽ được đại diện bởi một biến giả, có thể dẫn đến việc giảm chi phí lưu trữ hơn.
  + Dummy encoding giữ được thông tin về thứ tự của các giá trị phân loại. Ví dụ, biến giả cho giá trị "X" khác với biến giả cho giá trị "Y".
- Nhược điểm của dummy encoding là có thể làm tăng số lượng biến trong dữ liệu, vì mỗi giá trị phân loại sẽ được đại diện bởi một biến giả, dẫn đến việc tăng chi phí tính toán.

**Label Encoding**

- Label encoding là một kỹ thuật chuyển đổi các giá trị phân loại (categorical values) thành các giá trị số (numerical values). Mỗi giá trị phân loại được gán một số nguyên duy nhất, ví dụ như 'X', 'Y', 'Z', 'T' sẽ được gán cho là 0, 1, 2, 3
- Các mô hình học máy thường được thiết kế để làm việc với các dữ liệu số, vì vậy label encoding giúp chuyển đổi các giá trị phân loại (dạng string, object) thành các dữ liệu số, giúp các mô hình có thể hiểu và xử lý các giá trị này.Các mô hình có thể sử dụng label encoding:
  + Phân loại: logistic regression, decision tree, và random forest.
  + Hồi quy, chẳng hạn như linear regression, ridge regression, và lasso regression.
  + Clustering, chẳng hạn như k-means, hierarchical clustering, và density-based clustering.

In [11]:
categorical_cols = ['Development Name', 'Borough', 'Location', 'Vendor Name']
label_category_df = pd.DataFrame()
for col in categorical_cols:
    label_category_df[col] = category_df[col]
    label_category_df[col + '_Label_Encoded'], _ = pd.factorize(category_df[col]) 

label_category_df = label_category_df.drop_duplicates().reset_index(drop=True)
label_category_df

,Development Name,Development Name_Label_Encoded,Borough,Borough_Label_Encoded,Location,Location_Label_Encoded,Vendor Name,Vendor Name_Label_Encoded
0,ADAMS,0,BRONX,0,BLD 04,0,CONSOLIDATED EDISON COMPANY OF NY,0
1,ALBANY/ALBANY II,1,BROOKLYN,1,ALBANY BLD 04,1,NATIONAL GRID NYC,1
2,AMSTERDAM,2,MANHATTAN,2,BLD 02,2,CONSOLIDATED EDISON COMPANY OF NY,0
3,AMSTERDAM AVENUE,3,MANHATTAN,2,BLD 01,3,CONSOLIDATED EDISON COMPANY OF NY,0
4,ARMSTRONG I,4,BROOKLYN,1,BLD 02,2,NATIONAL GRID NYC,1
...,...,...,...,...,...,...,...,...
755,LONG ISLAND BAPTIST HOUSES,142,BROOKLYN,1,340 WILLIAMS AVENUE,121,NATIONAL GRID NYC,1
756,TWO BRIDGES URA (SITE 7),257,MANHATTAN,2,GEN,122,CONSOLIDATED EDISON COMPANY OF NY,0
757,ASTORIA,6,QUEENS,3,BLD 23,35,CONSOLIDATED EDISON COMPANY OF NY,0
758,EAST RIVER,298,MANHATTAN,2,GEN,122,CONSOLIDATED EDISON COMPANY OF NY,0


- Ưu điểm của label encoding:
  + Đơn giản và dễ thực hiện
  + Không làm thay đổi số lượng biến
  + Có thể sử dụng trong nhiều loại mô hình thống kê và học máy.
- Nhược điểm của label encoding:
  + Không giữ thông tin về thứ tự của các giá trị phân loại. Ví dụ, giá trị 0 cho "X" giống hệt với giá trị 0 cho "Y".
  + Nếu các giá trị phân loại có ý nghĩa thứ bậc, thì label encoding có thể làm mất thông tin về thứ bậc này. Ví dụ, nếu biến "Size" có các giá trị "M", "XXL", "L", và "XL", thì label encoding sẽ gán các giá trị 0, 1, 2, 3 cho các giá trị này. Cái bất lợi là giá trị 1 cho "XXL" không thể được so sánh trực tiếp với giá trị 2 cho "L", vì "XXL" có thứ bậc cao hơn "L".

**Count encoding**

- Count encoding là một kỹ thuật chuyển đổi các giá trị phân loại (categorical values) thành các giá trị số (numerical values). Mỗi giá trị phân loại được gán một số nguyên đại diện cho số lần xuất hiện của giá trị đó trong tập dữ liệu.
Ví dụ, nếu chúng ta có một cột phân loại với các giá trị "A", "B", "C", "D", và giá trị "A" xuất hiện 10 lần trong tập dữ liệu, giá trị "B" xuất hiện 20 lần, giá trị "C" xuất hiện 30 lần, giá trị "D" xuất hiện 50 lần thì count encoding sẽ gán các số nguyên 10, 20, 30, 50 cho các giá trị tương ứng.
- Count encoding có thể hữu ích cho các mô hình học máy có thể hiểu và xử lý thông tin về tần suất xuất hiện của các giá trị phân loại. Ví dụ, nếu chúng ta đang sử dụng một mô hình phân loại để dự đoán khả năng một khách hàng sẽ mua một sản phẩm, thì count encoding có thể được sử dụng để tính toán số lần khách hàng đó đã xem sản phẩm trong quá khứ. Số lần xem này có thể là một yếu tố dự đoán hữu ích cho mô hình.

In [12]:
count_encode_df = pd.DataFrame()
for col in categorical_cols:
    counts = category_df[col].value_counts().sort_values(ascending=False)
    count_encode_df[col] = category_df[col]
    count_encode_df[col + '_Count_Encoded'] = count_encode_df[col].map(counts)

count_encode_df = count_encode_df.drop_duplicates().reset_index(drop=True)
count_encode_df

,Development Name,Development Name_Count_Encoded,Borough,Borough_Count_Encoded,Location,Location_Count_Encoded,Vendor Name,Vendor Name_Count_Encoded
0,ADAMS,129,BRONX,22597,BLD 04,7988,CONSOLIDATED EDISON COMPANY OF NY,48298
1,ALBANY/ALBANY II,84,BROOKLYN,38437,ALBANY BLD 04,242,NATIONAL GRID NYC,42998
2,AMSTERDAM,139,MANHATTAN,23290,BLD 02,10138,CONSOLIDATED EDISON COMPANY OF NY,48298
3,AMSTERDAM AVENUE,45,MANHATTAN,23290,BLD 01,20255,CONSOLIDATED EDISON COMPANY OF NY,48298
4,ARMSTRONG I,400,BROOKLYN,38437,BLD 02,10138,NATIONAL GRID NYC,42998
...,...,...,...,...,...,...,...,...
755,LONG ISLAND BAPTIST HOUSES,233,BROOKLYN,38437,340 WILLIAMS AVENUE,18,NATIONAL GRID NYC,42998
756,TWO BRIDGES URA (SITE 7),120,MANHATTAN,23290,GEN,17,CONSOLIDATED EDISON COMPANY OF NY,48298
757,ASTORIA,217,QUEENS,4092,BLD 23,548,CONSOLIDATED EDISON COMPANY OF NY,48298
758,EAST RIVER,137,MANHATTAN,23290,GEN,17,CONSOLIDATED EDISON COMPANY OF NY,48298


- Ưu điểm của count encoding:
  + Count encoding giữ thông tin về tần suất xuất hiện của các giá trị phân loại, hữu ích cho các mô hình học máy có thể hiểu và xử lý thông tin này.
  + Đơn giản và dễ thực hiện
  + Không làm thay đổi số lượng biến trong dữ liệu.
- Nhược điểm của count encoding:
  + Có thể bị ảnh hưởng bởi kích thước của tập dữ liệu. Ví dụ, nếu chúng ta có một tập dữ liệu lớn, thì các giá trị phân loại xuất hiện nhiều lần sẽ có giá trị count encoding lớn hơn các giá trị phân loại xuất hiện ít lần. Điều này có thể dẫn đến sự không công bằng đối với các giá trị phân loại xuất hiện ít lần.
  + Có thể bị ảnh hưởng bởi sự phân bố không đồng đều của các giá trị phân loại: Ví dụ, nếu một giá trị phân loại xuất hiện rất nhiều lần, thì giá trị count encoding của giá trị đó sẽ rất cao, ngay cả khi giá trị đó không thực sự có liên quan đến biến mục tiêu.

### b. Các cột numeric

#### Có gặp giá trị bất thường (abnormal) không?
 Khi giá trị thấp hơn Q1 - 1.5*IQR hoặc cao hơn Q3* + 1.5IQR (với Q1, Q3 lần lượt là tứ phân vị thứ nhất, thứ ba, IQR = tứ phân vị thứ ba - tứ phân vị thứ nhất) thì giá trị đó là abnormal của cột tướng ứng.

In [13]:
numeric_df = eda_gas_df.loc[:, ['Current Charges', 'Consumption (Therms)']]
numeric_df = numeric_df.reset_index(drop=True)
numeric_df

,Current Charges,Consumption (Therms)
0,78292.97,136632.00
1,44335.21,153899.18
2,22034.86,76488.82
3,95456.57,169137.00
4,14607.73,26301.00
...,...,...
92194,1419.26,3615.00
92195,1036.26,1619.00
92196,2505.80,6768.00
92197,20992.58,50105.00


In [33]:
for col in numeric_df.columns:
    # Tính IQR rồi # Lấy các giá trị nằm ngoài phạm vi 1.5* IQR
    iqr = numeric_df[col].quantile(0.75) - numeric_df[col].quantile(0.25)
    abnormal_height = numeric_df[col].between(numeric_df[col].quantile(0.25) - 1.5 * iqr, numeric_df[col].quantile(0.75) + 1.5 * iqr)
    if True in abnormal_height.to_numpy():
        print("Cột", col, "có xuất hiện abnormal")


Cột Current Charges có xuất hiện abnormal
Cột Consumption (Therms) có xuất hiện abnormal


#### Scale the numeric column

Data normalization là một kỹ thuật xử lý dữ liệu nhằm biến đổi dữ liệu về một phạm vi giá trị nhất định, thường là từ 0 đến 1 hoặc từ -1 đến 1, từ đó có thể giúp cải thiện hiệu quả của các thuật toán học máy, giảm thiểu hiện tượng các biến có giá trị quá lớn lấn át các biến có giá trị bé hơn và giúp chúng ta hiểu rõ hơn về dữ liệu. Có nhiều phương pháp normalization khác nhau, nhưng phương pháp phổ biến nhất là min-max normalization, standard scale, robust scale.

**Standard Scale**

- Standard Scale là một kỹ thuật chuẩn hóa dữ liệu, trong đó các giá trị dữ liệu được chuyển đổi về một phân phối chuẩn có trung bình bằng 0 và độ lệch chuẩn bằng 1.
- Dùng StandardScaler cho các cột numeric bởi vì:
  + Các thuật toán học máy thường hoạt động tốt hơn trên dữ liệu có phân phối Guass. Chuẩn hóa dữ liệu giúp các thuật toán này hoạt động hiệu quả hơn bằng cách đưa dữ liệu về một phân phối chuẩn tắc.
  + Giảm ảnh hưởng của các outlier, bởi vì các outlier là những dữ liệu có giá trị bất thường so với phần còn lại của dữ liệu. Các outlier có thể làm sai lệch kết quả của các thuật toán học máy. Chuẩn hóa dữ liệu giúp giảm ảnh hưởng của các outlier bằng cách biến đổi chúng về gần phân phối chuẩn.

In [15]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)
standard_scale_df = pd.DataFrame(scaled_data, columns=['Current Charges_Standard_Scale', 'Consumption (Therms)_Standard_Scale'])
standard_scale_df

,Current Charges_Standard_Scale,Consumption (Therms)_Standard_Scale
0,2.668961,2.346788
1,1.341611,2.696895
2,0.469929,1.127332
3,3.339856,3.005855
4,0.179615,0.109730
...,...,...
92194,-0.335900,-0.350248
92195,-0.350870,-0.390719
92196,-0.293429,-0.286318
92197,0.429188,0.592377


Standard Scale giúp cải thiện độ chính xác của các mô hình hồi quy tuyến tính, hồi quy logistic, K-means clustering, support vector machines

**MinMax scale**

- Minmax là một phương pháp chuẩn hóa dữ liệu, trong đó giá trị của mỗi biến được biến đổi thành một giá trị mới nằm trong phạm vi từ 0 đến 1. Phương pháp này được thực hiện bằng cách tính toán giá trị tối thiểu (min) và giá trị tối đa (max) của biến, sau đó biến đổi mỗi giá trị của biến thành một giá trị mới bằng cách chia cho (max-min) và nhân với 1.
- Minmax cho các cột numeric giúp:
  + Tăng hiệu quả của các thuật toán học máy, bởi vì nhiều thuật toán học máy hoạt động tốt hơn trên dữ liệu có cùng thang đo và độ biến thiên. Chuẩn hóa dữ liệu bằng minmax giúp các biến numeric có cùng thang đo và độ biến thiên, từ đó giúp chúng đem lại kết quả chính xác hơn.
  + Giảm ảnh hưởng của các outlier - những dữ liệu có giá trị bất thường so với phần còn lại của dữ liệu. Chúng gây sai lệch kết quả của các thuật toán học máy. Chuẩn hóa dữ liệu bằng minmax giúp giảm ảnh hưởng của các outlier bằng cách biến đổi chúng về phạm vi giá trị từ 0 đến 1.
  + Giúp các biến numeric có thể so sánh được với nhau, từ đó giúp chúng ta hiểu rõ hơn về mối quan hệ giữa các biến.

In [16]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numeric_df)
minmax_scale_df = pd.DataFrame(scaled_data, columns=[ 'Current Charges_MinMax_Scale', 'Consumption (Therms)_MinMax_Scale'])
minmax_scale_df

,Current Charges_MinMax_Scale,Consumption (Therms)_MinMax_Scale
0,0.084319,0.058100
1,0.047747,0.065443
2,0.023731,0.032525
3,0.102804,0.071922
4,0.015732,0.011184
...,...,...
92194,0.001528,0.001537
92195,0.001116,0.000688
92196,0.002698,0.002878
92197,0.022608,0.021306


Minmax Scale có thể được sử dụng cho nhiều loại thuật toán học máy: hồi quy, phân lớp, gom cụm, mạng neural nhân tạo, ...

**Robust Scale**

- Robust Scaler là một kỹ thuật chuẩn hóa dữ liệu, trong đó giá trị của mỗi biến được biến đổi thành một giá trị mới nằm trong phạm vi từ 0 đến 1. Phương pháp này được thực hiện bằng cách tính toán phạm vi liên vị (IQR) của biến, sau đó biến đổi mỗi giá trị của biến thành một giá trị mới bằng cách chia cho IQR.
- Robust Scaler cho các cột numeric:
  + Chuẩn hóa dữ liệu bằng Robust Scaler giúp các biến numeric có cùng thang đo và độ biến thiên, từ đó giúp các thuật toán học máy hoạt động hiệu quả hơn.
  + Giảm ảnh hưởng của các outlier, vì chúng có thể làm sai lệch kết quả của các thuật toán học máy. Chuẩn hóa dữ liệu bằng Robust Scaler giúp giảm ảnh hưởng của các outlier bằng cách chỉ tính toán phạm vi liên vị, trong đó các outlier được loại bỏ.

In [17]:
scaler = RobustScaler()
scaled_data = scaler.fit_transform(numeric_df)
robust_scale_df = pd.DataFrame(scaled_data, columns=['Current Charges_Robust_Scale', 'Consumption (Therms)_Robust_Scale'])
robust_scale_df

,Current Charges_Robust_Scale,Consumption (Therms)_Robust_Scale
0,13.037723,9.836272
1,7.284306,11.101626
2,3.505988,5.428927
3,15.945729,12.218267
4,2.247620,1.751120
...,...,...
92194,0.013115,0.088670
92195,-0.051776,-0.057599
92196,0.197206,0.319724
92197,3.329396,3.495498


- Robust Scaler và MinMaxScaler đều là các kỹ thuật chuẩn hóa dữ liệu, nhưng có một số điểm khác biệt cơ bản giữa hai phương pháp này.
  + Cách tính toán: MinMaxScaler sử dụng giá trị tối thiểu (min) và giá trị tối đa (max) của biến để tính toán phạm vi, trong khi Robust Scaler sử dụng phạm vi liên vị (IQR) của biến.
  + Ảnh hưởng của các outlier: MinMaxScaler í nhiều vẫn bị ảnh hưởng bởi các outlier, trong khi Robust Scaler gần như không bị ảnh hưởng bởi các outlier.
- Vì vậy, Robust Scaler thường được sử dụng trong các trường hợp dữ liệu có các outlier, trong khi MinMaxScaler thường được sử dụng trong các trường hợp dữ liệu không có hoặc có rất ít outlier.

#### Data Reduction - giảm chiều dữ liệu
- Data reduction là quá trình giảm dung lượng dữ liệu cần thiết để lưu trữ và xử lý, có thể được thực hiện thông qua nhiều phương pháp khác nhau:
  + Feature selection: Lựa chọn các features quan trọng nhất và loại bỏ các features không quan trọng.
  + Data compression: Thu gọn dữ liệu bằng cách giảm độ chính xác hoặc độ phức tạp của dữ liệu.
  + Data summarization: Tổng hợp dữ liệu bằng cách tạo ra các đại diện ngắn gọn và súc tích của dữ liệu.
- Data reduction có nhiều tác dụng, bao gồm:
  + Giảm dung lượng dữ liệu cần thiết để lưu trữ và xử lý, từ đó tiết kiệm chi phí và thời gian.
  + Tăng hiệu quả xử lý dữ liệu bằng cách giảm thời gian và tài nguyên cần thiết.
  + Tăng khả năng hiểu dữ liệu bằng cách làm cho dữ liệu dễ dàng truy cập và phân tích hơn.
- Data reduction thường được sử dụng trong các lĩnh vực ứng dụng như:
  + Học máy: cải thiện hiệu quả của các thuật toán học máy bằng cách giảm số lượng features cần thiết để đào tạo mô hình.
  + Phân tích dữ liệu: tăng khả năng phân tích dữ liệu bằng cách làm cho dữ liệu dễ dàng truy cập và phân tích hơn.
  + Kho dữ liệu: giảm chi phí và thời gian lưu trữ dữ liệu.

**Principal Component Analysis (PCA)**

Phân tích thành phần chính PCA là một kỹ thuật data reduction, có thể giúp giảm số lượng features cần thiết để lưu trữ và xử lý dữ liệu. PCA hoạt động bằng cách tìm các thành phần chính của dữ liệu. Các thành phần chính là các biến mới được tạo ra từ các biến ban đầu. Các thành phần chính được sắp xếp theo mức độ quan trọng, với thành phần chính đầu tiên chứa nhiều thông tin nhất.

In [18]:
pca = PCA(n_components=None, svd_solver="auto")
numeric_df_pca = pca.fit_transform(numeric_df)
numeric_df_pca = pd.DataFrame(numeric_df_pca, columns=['Current Charges_PCA', 'Consumption (Therms)_PCA'])
# In kết quả
numeric_df_pca

,Current Charges_PCA,Consumption (Therms)_PCA
0,134034.314963,9665.460170
1,134393.191175,-28428.583685
2,55143.611816,-13965.747970
3,170781.176027,10577.815342
4,6891.016688,1708.007157
...,...,...
92194,-19293.552690,-12.069421
92195,-21251.563885,532.813281
92196,-15986.357883,-441.534996
92197,31050.353385,-3162.637143


#### Gom cụm dữ liệu 

**K- means clustering**
- K-means là một thuật toán phân cụm không giám sát, được sử dụng để phân chia dữ liệu thành các cụm dựa trên tính tương đồng của các điểm dữ liệu. K-means hoạt động bằng cách khởi tạo một số cụm ngẫu nhiên, sau đó lặp lại các bước sau cho đến khi đạt được điểm dừng:
  + Gán mỗi điểm dữ liệu vào cụm gần nhất với nó.
  + Tính trung tâm mới cho mỗi cụm.
  + Các trung tâm cụm mới được tính bằng cách lấy trung bình của các điểm dữ liệu trong mỗi cụm. Quá trình này được lặp lại cho đến khi các trung tâm cụm không thay đổi nữa.
- K-means có nhiều tác dụng, bao gồm:
  + Phân cụm dữ liệu dựa trên tính tương đồng của các điểm dữ liệu, dễ dàng trong việc tìm các nhóm khách hàng, phân loại đối tượng hoặc khám phá dữ liệu.
  + Giảm chiều dữ liệu bằng cách sử dụng các trung tâm cụm làm đại diện cho các điểm dữ liệu, sử dụng để cải thiện hiệu quả của các thuật toán học máy.
  + Tìm các cấu trúc trong dữ liệu, chẳng hạn như các nhóm dữ liệu phân tán hoặc các đối tượng có hình dạng đặc biệt.

In [19]:
kmeans = KMeans(n_clusters=3, n_init=1)
df_clustered = kmeans.fit_predict(numeric_df)
kmeans_df = numeric_df.copy()
kmeans_df['Cluster'] = df_clustered
kmeans_df

,Current Charges,Consumption (Therms),Cluster
0,78292.97,136632.00,2
1,44335.21,153899.18,2
2,22034.86,76488.82,2
3,95456.57,169137.00,1
4,14607.73,26301.00,0
...,...,...,...
92194,1419.26,3615.00,0
92195,1036.26,1619.00,0
92196,2505.80,6768.00,0
92197,20992.58,50105.00,2


**Rời rạc hóa**
- Discretized là một thuật ngữ chỉ việc biến đổi, gom cụm dữ liệu liên tục thành dữ liệu rời rạc. Điều này có thể được thực hiện bằng cách sử dụng các phương pháp khác nhau, chẳng hạn như:
  + Binning: Chia dữ liệu thành các khoảng giá trị rời rạc.
  + Thresholding: Sử dụng ngưỡng để phân loại dữ liệu thành các giá trị rời rạc.
  + Quantization: Sử dụng các hàm quantization để biến đổi dữ liệu liên tục thành các giá trị rời rạc.
- Tác dụng của discretized
  + Tăng hiệu quả của các thuật toán học máy, vì nhiều thuật toán học máy hoạt động tốt hơn trên dữ liệu rời rạc.
  + Giảm ảnh hưởng của các outlier: Các outlier có thể làm sai lệch kết quả của các thuật toán học máy. Discretized có thể giúp giảm ảnh hưởng của các outlier bằng cách gán chúng cho các giá trị rời rạc.
- Các thuật toán học máy dùng discretized
  + Phân loại: k-nearest neighbors và decision trees.
  + Các thuật toán khối lượng công việc, chẳng hạn như scheduling và resource allocation.
  + Các thuật toán tìm kiếm, chẳng hạn như nearest neighbors và k-means.

In [20]:
discretized_df = pd.DataFrame()
for col in numeric_df.columns:
    discretized_df[col] = numeric_df[col]
    discretized_df[col + "_Discretized"] = pd.qcut(numeric_df[col], 3)
discretized_df

,Current Charges,Current Charges_Discretized,Consumption (Therms),Consumption (Therms)_Discretized
0,78292.97,"(3352.53, 928530.45]",136632.00,"(6590.333, 2351667.0]"
1,44335.21,"(3352.53, 928530.45]",153899.18,"(6590.333, 2351667.0]"
2,22034.86,"(3352.53, 928530.45]",76488.82,"(6590.333, 2351667.0]"
3,95456.57,"(3352.53, 928530.45]",169137.00,"(6590.333, 2351667.0]"
4,14607.73,"(3352.53, 928530.45]",26301.00,"(6590.333, 2351667.0]"
...,...,...,...,...
92194,1419.26,"(620.92, 3352.53]",3615.00,"(1218.0, 6590.333]"
92195,1036.26,"(620.92, 3352.53]",1619.00,"(1218.0, 6590.333]"
92196,2505.80,"(620.92, 3352.53]",6768.00,"(6590.333, 2351667.0]"
92197,20992.58,"(3352.53, 928530.45]",50105.00,"(6590.333, 2351667.0]"


**Binning**

- Binning là một kỹ thuật xử lý dữ liệu nhằm biến đổi dữ liệu liên tục thành dữ liệu rời rạc - chia dữ liệu thành các khoảng giá trị rời rạc, được gọi là các bin.
  + Fixed-width binning: Các bin có cùng chiều rộng.
  + Equal-frequency binning: Các bin có cùng số lượng quan sát.
  + Quantile binning: Các bin chứa số lượng quan sát bằng nhau theo thứ tự phân phối.

- Tác dụng của binning
  + Tăng hiệu quả của các thuật toán học máy - hoạt động tốt hơn trên dữ liệu rời rạc.
  + Giảm ảnh hưởng của các outlier bằng cách gán chúng cho các bin riêng biệt.
  + Tăng khả năng hiểu dữ liệu - giúp chúng ta hiểu rõ hơn về dữ liệu bằng cách nhóm các quan sát có giá trị tương tự lại với nhau.

In [21]:
binning_df = pd.DataFrame()
for col in numeric_df.columns:
    binning_df[col] = numeric_df[col]
    binning_df[col + "_Binning"] = pd.cut(numeric_df[col], [0,
                                                            np.quantile(numeric_df[col], 0.25),
                                                            np.quantile(numeric_df[col], 0.5),
                                                            np.quantile(numeric_df[col], 0.75),
                                                            np.inf])
binning_df

,Current Charges,Current Charges_Binning,Consumption (Therms),Consumption (Therms)_Binning
0,78292.97,"(6341.27, inf]",136632.00,"(14441.125, inf]"
1,44335.21,"(6341.27, inf]",153899.18,"(14441.125, inf]"
2,22034.86,"(6341.27, inf]",76488.82,"(14441.125, inf]"
3,95456.57,"(6341.27, inf]",169137.00,"(14441.125, inf]"
4,14607.73,"(6341.27, inf]",26301.00,"(14441.125, inf]"
...,...,...,...,...
92194,1419.26,"(1341.85, 6341.27]",3615.00,"(2405.0, 14441.125]"
92195,1036.26,"(439.08, 1341.85]",1619.00,"(795.0, 2405.0]"
92196,2505.80,"(1341.85, 6341.27]",6768.00,"(2405.0, 14441.125]"
92197,20992.58,"(6341.27, inf]",50105.00,"(14441.125, inf]"


So sánh binning và k-means:
- Mục tiêu
  + Binning chia dữ liệu thành các khoảng giá trị rời rạc, được gọi là các bin.
  + K-means phân dữ liệu thành k cụm, với mỗi cụm có trung tâm là một điểm dữ liệu.
- Cách thức hoạt động
  + Binning được thực hiện bằng cách chia dữ liệu thành các khoảng giá trị rời rạc, với số lượng bin và kích thước bin được xác định trước.
  + K-means hoạt động theo các bước sau:
    - Chọn ngẫu nhiên k điểm dữ liệu làm trung tâm của các cụm.
    - Gán mỗi điểm dữ liệu cho cụm có trung tâm gần nhất với nó.
    - Tính toán lại trung tâm của mỗi cụm.
    - Lặp lại các bước 2 và 3 cho đến khi các trung tâm của các cụm không thay đổi nữa.
- Ưu điểm
  + Binning đơn giản và dễ hiểu, có thể được sử dụng cho nhiều loại dữ liệu khác nhau và giúp giảm ảnh hưởng của các outlier.
  + K-means tìm ra các cụm có hình dạng bất kỳ, phân cụm dữ liệu có nhiễu.
- Nhược điểm
  + Binning có thể làm mất thông tin của dữ liệu gốc, khó chọn số lượng bin và kích thước bin phù hợp.
  + K-means có thể bị ảnh hưởng bởi điểm khởi đầu, mất nhiều thời gian để chạy, đặc biệt là khi số lượng điểm dữ liệu lớn.
